In [1]:
import os, sys

sys.path.append(os.path.abspath(os.path.join("..")))

import jax

jax.config.update("jax_enable_x64", True)

from jaxtyping import Array
import jax.numpy as jnp
from tx import TransformerWithHooks

In [2]:
reference_gpt2 = TransformerWithHooks.from_pretrained("gpt2", decode=True)


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
2023-10-15 04:51:39.687942: W external/tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 736.19MiB (rounded to 771948032)requested by op 
2023-10-15 04:51:39.688796: W external/tsl/tsl/framework/bfc_allocator.cc:497] ******____*********************************************************************_____________________
2023-10-15 04:51:39.689034: E external/xla/xla/pjrt/pjrt_stream_executor_client.cc:2717] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 771947904 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:         8B
              constant allocation:         0B
        maybe_live_out allocation:  294.47MiB
     preallocated temp allocation:  736.19MiB
  preallocated temp fragmentation:  

XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 771947904 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:         8B
              constant allocation:         0B
        maybe_live_out allocation:  294.47MiB
     preallocated temp allocation:  736.19MiB
  preallocated temp fragmentation:         0B (0.00%)
                 total allocation:    1.01GiB
              total fragmentation:  147.24MiB (14.29%)
Peak buffers:
	Buffer 1:
		Size: 294.47MiB
		Operator: op_name="jit(_normal)/jit(main)/jit(_normal_real)/mul" source_file="/home/alexj/workspace/tx/env/lib/python3.11/site-packages/flax/core/scope.py" source_line=983
		XLA Label: fusion
		Shape: f64[768,50257]
		==========================

	Buffer 2:
		Size: 147.24MiB
		Operator: op_name="jit(_normal)/jit(main)/jit(_normal_real)/jit(_uniform)/threefry2x32" source_file="/home/alexj/workspace/tx/env/lib/python3.11/site-packages/flax/core/scope.py" source_line

In [4]:
reference_text = "Hello, I am"
tokens: Array = reference_gpt2.to_tokens(reference_text, prepend_bos=True)
print(tokens)


[50256 15496    11   314   716]


In [4]:
# print(reference_gpt2.to_str(tokens), end="", flush=True)
# for i in range(tokens.shape[0]):
#     # Pass sequence through the model to get new output
#     logits, _ = reference_gpt2(tokens[: i + 1])
#     # Get the predicted token at the end of our sequence
#     next_token = jnp.argmax(logits, axis=-1)[-1]
#     # Decode and print the result
#     next_char = reference_gpt2.to_str(next_token)
#     print(f"next_token[{i}]: {next_char}")


In [5]:
logits, _ = reference_gpt2(tokens)
next_token = jnp.argmax(jax.nn.softmax(logits), axis=-1)[-1]
next_char = reference_gpt2.to_str(next_token)
print(next_char)

 a


In [6]:
cur_tokens = tokens
print(reference_gpt2.to_str(cur_tokens), end="", flush=True)
for i in range(10):
    # Pass sequence through the model to get new output
    logits, _ = reference_gpt2(cur_tokens)
    # Get the predicted token at the end of our sequence
    next_token = jnp.argmax(logits, axis=-1)[-1]
    # Decode and print the result
    next_char = reference_gpt2.to_str(next_token)
    print(next_char, end="", flush=True)
    # Define new input sequence, by appending the previously generated token
    cur_tokens = jnp.append(cur_tokens, next_token)


<|endoftext|>Hello, I am a student at the University of California, Berkeley.